## Практическая работа

В этой практической работе четыре обязательные задачи.

*Обязательные задачи* нужно сделать для того, чтобы проверить, что вы действительно усвоили материал модуля. Сдайте их на проверку.

Удачи!

Цели практической работы: 


1.   Потренироваться в обучении моделей деревьев решений.
2.   Потренироваться в обучении моделей случайного леса. 
3.   Научиться оценивать качество моделей с помошью Accuracy и confusion matrix.
4.   Научиться увеличивать качество моделей с помощью тюнинга параметров.




Что оценивается:

*   Все пункты и критерия приёмки задания выполнены.
*   Удаление колонок по результатам feature_importance и определения типов производится с помощью кода, а не перечислением их вручную.




## Обязательные задачи

### Описание датасета:
- `id`: идентификатор записи;
- `is_manufacturer_name`: признак производителя автомобиля;

- `region_*`: регион;
- `x0_*`: тип топлива;
- `manufacturer_*`: производитель;
- `short_model_*`: сокращённая модель автомобиля;
- `title_status_*`: статус;
- `transmission_*`: коробка передач;
- `state_*`: штат;
- `age_category_*`: возрастная категория автомобиля;

- `std_scaled_odometer`: количество пройденных миль (после стандартизации);
- `year_std`: год выпуска (после стандартизации);
- `lat_std`: широта (после стандартизации);
- `long_std`: долгота (после стандартизации);
- `odometer/price_std`: отношение стоимости к пробегу автомобиля (после стандартизации);
- `desc_len_std`: количество символов в тексте объявления о продаже (после стандартизации);
- `model_in_desc_std`: количество наименований модели автомобиля в тексте объявления о продаже (после стандартизации);
- `model_len_std`: длина наименования автомобиля (после стандартизации);
- `model_word_count_std`: количество слов в наименовании автомобиля (после стандартизации);
- `month_std`: номер месяца размещения объявления о продаже автомобиля (после стандартизации);
- `dayofweek_std`: день недели размещения объявления о продаже автомобиля (после стандартизации);
- `diff_years_std`: количество лет между годом производства автомобиля и годом размещения объявления о продаже автомобиля (после стандартизации);

- `price`: стоимость;
- `price_category`: категория цены.

0. *Подготовка базовой модели*

Обучите простую модель классификации с помощью DecisionTreeClassifier на данных из датасета vehicles_dataset_prepared.csv. Для этого сделайте следующие шаги:

1. Обучите модель дерева решений с зафиксированным random_state на тренировочной выборке.
2. Сделайте предикт на тестовой выборке.
3. Замерьте точность на тестовой выборке и выведите матрицу ошибок. 
4. Удалите фичи с нулевыми весами по feature_importance из тренировочной и тестовой выборок.
5. Заново обучите модель и измерьте качество.

In [1]:
import pandas as pd
from envs.ml.DLLs.pyexpat import features
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, confusion_matrix 

In [2]:
df = pd.read_csv('vehicles_dataset_prepared.csv')

df_prepared = df.copy()
df_prepared = df_prepared.drop(['price', 'odometer/price_std'], axis=1)

x = df_prepared.drop(['price_category'], axis=1)
y = df_prepared['price_category']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

model = DecisionTreeClassifier()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

In [3]:
acc = accuracy_score(y_test, y_pred)
print(acc)
confusion_matrix(y_test, y_pred)

0.6701316701316701


array([[728,  55, 214],
       [ 45, 693, 215],
       [188, 235, 513]], dtype=int64)

In [4]:
features_rate = [[feature, rate] for feature, rate in zip(x_train.columns, model.feature_importances_)]
features_rate = pd.DataFrame(features_rate, columns=['feature', 'importance'])
useless_features = features_rate[features_rate.importance == 0.00]

x_train_cuted = x_train.drop(useless_features.feature, axis=1)
x_test_cuted = x_test.drop(useless_features.feature, axis=1)

In [5]:
model2 = DecisionTreeClassifier()
model2.fit(x_train_cuted, y_train)
y_pred = model2.predict(x_test_cuted)

In [6]:
acc = accuracy_score(y_test, y_pred)
print(acc)
confusion_matrix(y_test, y_pred)

0.6763686763686764


array([[738,  56, 203],
       [ 44, 696, 213],
       [192, 226, 518]], dtype=int64)

1. *Подготовка модели случайного леса*

Обучите простую модель классификации с помощью RandomForestClassifier. Для этого сделайте следующие шаги:
1. На новых урезанных семплах тренировочной и тестовой выборок обучите модель случайного леса с зафиксированным random_state=50. 

2. Сделайте предикт и посчитайте точность модели и матрицу ошибок. Сравните с предыдущей моделью дерева решений. Есть ли случаи, когда модель из пункта 1 отрабатывает лучше, чем модель случайного леса?

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=50)

model3 = RandomForestClassifier()
model3.fit(x_train, y_train)

y_pred = model3.predict(x_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

confusion_matrix(y_test, y_pred)

0.7557172557172557


array([[783,  42, 121],
       [ 12, 818, 145],
       [171, 214, 580]], dtype=int64)

2. *Тюнинг модели случайного леса*

Увеличьте точность модели на тестовом датасете RandomForestClassifier c помощью тюнинга параметров. 

Параметры, которые можно настраивать для увеличения точности:

```
    `bootstrap'
    'max_depth'
    'max_features'
    'min_samples_leaf'
    'min_samples_split'
    'random_state'
    'n_estimators'

```



С описанием каждого из параметров можно ознакомиться в документации:


https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

Задание засчитывается, если значение метрики строго выше 0,76 на тестовом датасете.

In [8]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
   'bootstrap': [True, False],
   'max_depth': [None] + list(range(500, 1500, 100)),
   'min_samples_split': list(range(2, 50)),
   'n_estimators': list(range(500, 1500, 100)),
}
randomized_search_rf = RandomizedSearchCV(
   estimator=model3,
   param_distributions=param_grid,
   n_iter=50,
   scoring='accuracy',
   verbose=1,
   n_jobs=-1
)

randomized_search_rf.fit(x_train, y_train)

best_params = randomized_search_rf.best_params_
print(best_params)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'n_estimators': 1000, 'min_samples_split': 3, 'max_depth': 700, 'bootstrap': False}


In [8]:
best_params = {'n_estimators': 1000, 'min_samples_split': 3, 'max_depth': 700, 'bootstrap': False}

In [10]:
model3 = RandomForestClassifier(**best_params)
model3.fit(x_train, y_train)

y_pred = model3.predict(x_test)
acc = accuracy_score(y_test, y_pred)
print(acc)
confusion_matrix(y_test, y_pred)

0.773042273042273


array([[795,  37, 114],
       [ 14, 813, 148],
       [138, 204, 623]], dtype=int64)

3. *Анализ влияния фичей на модель*

Во всех задачах до вы работали над подготовленным датасетом, где фичи были заранее извлечены из текстовых переменных, отскейлены и пропущены через One Hot Encoder. Сравним, какой была бы предсказательная способность модели, если бы мы использовали только сырые данные из исходного датасета. Для этого проделайте следующие шаги:

1. Загрузите датасет `vehicles_dataset_old.csv`.
2. Удалите из него переменную `price` и все строковые колонки. Дерево решений и случайный лес не умеют самостоятельно работать со строковыми значениями.
3. Сформируйте x_train и x_test так же, как они были сформированы в предыдущих заданиях.
4. Обучите свою лучшую модель случайного леса на новых данных и замерьте качество. Убедитесь, что оно ухудшилось.
5. Найдите три фичи, которые лучшим образом повлияли на предсказательную способность модели.

In [9]:
df_old = pd.read_csv(r'.\vehicles_dataset_old.csv')
df_old.head()

id  \
0  7308295377   
1  7316380095   
2  7313733749   
3  7308210929   
4  7303797340   

                                                                                               url  \
0  https://chattanooga.craigslist.org/ctd/d/chattanooga-2020-ram-2500-crew-cab-big/7308295377.html   
1              https://newjersey.craigslist.org/ctd/d/carlstadt-2016-ford-explorer/7316380095.html   
2                https://reno.craigslist.org/ctd/d/atlanta-2017-vw-volkswagen-golf/7313733749.html   
3   https://fayetteville.craigslist.org/ctd/d/raleigh-2013-toyota-rav4-xle-4dr-suv/7308210929.html   
4   https://knoxville.craigslist.org/ctd/d/knoxville-2012-bmw-series-128i-coupe-2d/7303797340.html   

         region                           region_url  price  year  \
0   chattanooga   https://chattanooga.craigslist.org  54990  2020   
1  north jersey     https://newjersey.craigslist.org  16942  2016   
2  reno / tahoe          https://reno.craigslist.org  35590  2017   
3  fayetteville  https://fayetteville.craigslist.org  14500  2013   
4     knoxville     https://knoxville.craigslist.org  14590  2012   

  manufacturer                   model    fuel  odometer title_status  \
0          ram  2500 crew cab big horn  diesel     27442        clean   
1         ford    explorer 4wd 4dr xlt   other     60023        clean   
2   volkswagen        golf r hatchback     gas     14048        clean   
3       toyota                    rav4     gas    117291        clean   
4          bmw  1 series 128i coupe 2d   other     80465        clean   

  transmission  \
0        other   
1    automatic   
2        other   
3    automatic   
4        other   

                                                             image_url  \
0  https://images.craigslist.org/00N0N_1xMPvfxRAIdz_0gw0co_600x450.jpg   
1  https://images.craigslist.org/00x0x_26jl9F0cnLvz_0ak06T_600x450.jpg   
2  https://images.craigslist.org/00y0y_eeZjWeiSfbUz_0gw0co_600x450.jpg   
3  https://images.craigslist.org/00606_iGe5iXidibAz_0hd0ew_600x450.jpg   
4  https://images.craigslist.org/00F0F_5UAXmOzC188z_0gw0co_600x450.jpg   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [10]:
str_columns = [column for column in df_old.columns if df_old[column].dtype == 'O']
df_old = df_old.drop(str_columns, axis=1)
x = df_old.drop(['price'], axis=1)
y = df_old['price']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [125]:
model4 = RandomForestClassifier(**best_params)
model4.fit(x_train[:1000], y_train[:1000])

RandomForestClassifier(bootstrap=False, max_depth=900, n_estimators=1000)

In [126]:
y_pred = model4.predict(x_test)
acc = accuracy_score(y_test, y_pred)
print(acc)
confusion_matrix(y_test, y_pred)

0.020443520443520442


array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [130]:
features_rate = [[feature, rate] for feature, rate in zip(x_train.columns, model4.feature_importances_)]
features_rate = pd.DataFrame(features_rate, columns=['feature', 'importance'])
features_rate.sort_values('importance', ascending=False)[:3]

,feature,importance
2,odometer,0.218652
0,id,0.213666
3,lat,0.211136
